## Importing necessary libraries

In [ ]:
import scrapy
from scrapy.crawler import CrawlerProcess
import requests

import pandas as pd

## Creating and runnign the Crawler

In [ ]:
# Create the Spider class
class FC_Spider(scrapy.Spider):
    name = "fc_spider"
    empty_page_checker_1 = False
    empty_page_checker_2 = False
    itter = 1

    # start_requests method
    def start_requests(self):
        
        for status in ["true","mostly-true","half-true","barely-true","false","pants-fire"]:
           
            self.empty_page_checker_1 = False
            self.empty_page_checker_2 = False
            self.itter = 1

            while not self.empty_page_checker_1 and not self.empty_page_checker_2:
                
                url = "https://www.politifact.com/factchecks/list/?page={}&ruling={}".format(self.itter, status)
                self.empty_page_checker_1 = not requests.get(url).ok

                if not self.empty_page_checker_1:
                    yield scrapy.Request(url = url
                                        ,callback = self.parse_list
                                        ,cb_kwargs = dict(status=status))
    
    # First parsing method
    def parse_list(self, response, status):
        
        self.empty_page_checker_2 = response.xpath('//h2[contains(text(), "No Results found")]').extract_first() != None
        
        if not self.empty_page_checker_2:
        
            self.itter += 1
            post_blocks = response.css('div.m-statement__quote')
            post_links = post_blocks.xpath('./a/@href').extract()

            for url in post_links:
                yield response.follow(url = url
                                     ,callback = self.parse_post
                                     ,cb_kwargs=dict(status=status))
            

    # Second parsing method
    def parse_post(self, response, status):
        # Extracting necessary info from page
        next_row_id = len(fc_df)
        
        fc_df.loc[next_row_id,'post'] = response.css('div.m-statement__quote::text').extract_first().strip()
        fc_df.loc[next_row_id,'author'] = response.css('a.m-statement__name::text').extract_first().strip()
        fc_df.loc[next_row_id,'desc'] = response.css('div.m-statement__desc::text').extract_first().strip()
        fc_df.loc[next_row_id,'status'] = status

fc_df = pd.DataFrame(columns=['post', 'author', 'desc', 'status'])

# Run the Spider
process = CrawlerProcess()
process.crawl(FC_Spider)
process.start()

## Exporting the data as intermediate result

In [ ]:
fc_df.to_csv('fc_df.csv', encoding='utf-8')